As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to:

Scrape data from destinations
Get weather data from each destination
Get hotels' info about each destination
Store all the information above in a data lake
Extract, transform and load cleaned data from your datalake to a data warehouse

In [11]:
# import libraries
import requests
import json
import pandas as pd
from statistics import mean
import datetime
import plotly.express as px
import plotly.io as pio
import boto3
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

Marketing team wants to focus first on the best cities to travel to in France. According One Week In.com here are the top-35 cities to visit in France:

In [50]:
# list of cities

top_35_cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [51]:
# scraping geolocation of cities
# Use https://nominatim.org/ 

#  https://nominatim.openstreetmap.org/search?<params>
endpoint = "https://nominatim.openstreetmap.org"

response =  requests.get(endpoint)
print("Response code:",response,"\n \n")

Response code: <Response [200]> 
 



In [52]:
# example to Ariege
response =  requests.get("https://nominatim.openstreetmap.org/search?q=Ariege&format=json")
print("Response code:",response,"\n \n")
print("Response data:\n")
response.json()

Response code: <Response [200]> 
 

Response data:



[{'place_id': 297389050,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 7439,
  'boundingbox': ['42.5732416', '43.3162514', '0.8267506', '2.1758135'],
  'lat': '42.9455368',
  'lon': '1.4065544156065486',
  'display_name': 'Ariège, Occitanie, France métropolitaine, France',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.6009114788084189,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'},
 {'place_id': 299422341,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 13625918,
  'boundingbox': ['51.1624195', '51.199616', '-56.0111241', '-55.9723346'],
  'lat': '51.18111155',
  'lon': '-55.98447423947262',
  'display_name': 'Ariege (Belvy) Bay, Main Brook, Newfoundland, Newfoundland and Labrador, Canada',
  'class': 'natural',
  'type': 'bay',
  'importance': 0.299999999999

In [53]:
# intialization empty dataframe
df_cities = pd.DataFrame(columns = ['city_id', 'city' , 'latitude', 'longitude'])

In [54]:
# browse the list of 35 cities
for counter, city in enumerate(top_35_cities):
    response =  requests.get(f"https://nominatim.openstreetmap.org/search?q={city}&format=json")
    # Complete the dataframe
    df_cities.loc[counter] = [response.json()[0]['place_id'],
                              city,
                              response.json()[0]['lat'],
                              response.json()[0]['lon']]

In [55]:
df_cities

,city_id,city,latitude,longitude
0,156094680,Mont Saint Michel,48.6359541,-1.511459954959514
1,297756747,St Malo,48.649518,-2.0260409
2,297981358,Bayeux,49.2764624,-0.7024738
3,298137491,Le Havre,49.4938975,0.1079732
4,297518815,Rouen,49.4404591,1.0939658
5,297417241,Paris,48.8588897,2.3200410217200766
6,297534793,Amiens,49.8941708,2.2956951
7,297472400,Lille,50.6365654,3.0635282
8,297508568,Strasbourg,48.584614,7.7507127
9,120791766,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195


In [56]:
len(df_cities)

35

In [57]:
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   city_id    35 non-null     int64 
 1   city       35 non-null     object
 2   latitude   35 non-null     object
 3   longitude  35 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.4+ KB


In [58]:
# scrap the weather
# Use https://openweathermap.org/appid 

# some information about the weather for the 35 cities
# https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}

# import libraries

from credentials import MY_APIKEY

# API key
APPID = MY_APIKEY

EXCLUDE = 'current,minutely,hourly'

In [59]:
endpoint = "https://api.openweathermap.org/data/2.5/onecall?"

# example
params = {'exclude' : EXCLUDE ,
          'appid' : APPID ,
          'lat' : '48.6359541' ,
          'lon' : -1.511459954959514,
          'units' : 'metric'}

response =  requests.get(endpoint, params = params)
print("Response code:",response,"\n \n")
response.json()

Response code: <Response [200]> 
 



{'lat': 48.636,
 'lon': -1.5115,
 'timezone': 'Europe/Paris',
 'timezone_offset': 7200,
 'daily': [{'dt': 1665745200,
   'sunrise': 1665728646,
   'sunset': 1665767984,
   'moonrise': 1665775920,
   'moonset': 1665747600,
   'moon_phase': 0.65,
   'temp': {'day': 16.39,
    'min': 12.97,
    'max': 16.41,
    'night': 16.41,
    'eve': 15.48,
    'morn': 13.23},
   'feels_like': {'day': 16.57, 'night': 16.61, 'eve': 15.56, 'morn': 13.12},
   'pressure': 1012,
   'humidity': 95,
   'dew_point': 15.59,
   'wind_speed': 5.48,
   'wind_deg': 207,
   'wind_gust': 11.58,
   'weather': [{'id': 501,
     'main': 'Rain',
     'description': 'moderate rain',
     'icon': '10d'}],
   'clouds': 100,
   'pop': 1,
   'rain': 9.44,
   'uvi': 0.56},
  {'dt': 1665831600,
   'sunrise': 1665815136,
   'sunset': 1665854266,
   'moonrise': 1665864840,
   'moonset': 1665837780,
   'moon_phase': 0.68,
   'temp': {'day': 15.65,
    'min': 13.03,
    'max': 17.77,
    'night': 14.17,
    'eve': 15.93,
    'mor

In [60]:
# date of extraction
date = response.json()['daily'][0]['dt']
readable = datetime.datetime.fromtimestamp(date).isoformat()
readable

'2022-10-14T13:00:00'

In [61]:
df_cities

,city_id,city,latitude,longitude
0,156094680,Mont Saint Michel,48.6359541,-1.511459954959514
1,297756747,St Malo,48.649518,-2.0260409
2,297981358,Bayeux,49.2764624,-0.7024738
3,298137491,Le Havre,49.4938975,0.1079732
4,297518815,Rouen,49.4404591,1.0939658
5,297417241,Paris,48.8588897,2.3200410217200766
6,297534793,Amiens,49.8941708,2.2956951
7,297472400,Lille,50.6365654,3.0635282
8,297508568,Strasbourg,48.584614,7.7507127
9,120791766,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195


In [63]:
# params
params = {'exclude' : EXCLUDE ,
          'appid' : APPID ,
          'units' : 'metric'}

# initialization of an empty dataframe
df_cities_weather = pd.DataFrame(columns = ['city_id', 'city','latitude','longitude','temperature', 'clear_weather'])

# browse the list of 35 cities by latitude and longitude
for counter, (id, city, lat, lon) in enumerate(zip(df_cities['city_id'],
                                                   df_cities['city'],
                                                   df_cities['latitude'],
                                                   df_cities['longitude'])):
                                                   
    # latitude and longitude of each city
    localpoint = endpoint + f"lat={lat}&lon={lon}"
    response =  requests.get(localpoint, params = params)
    
    # Retrieves weather information for the next 7 days
    temp = []
    clear_weather = []
    for i in range(1, len(response.json()['daily'])):
        # Day temperature
        temp.append((response.json()['daily'][i]['temp']['day']))
        # clear weather: 100 - Cloudiness, %
        clear_weather.append(100 - (response.json()['daily'][i]['clouds']))
        
    # Complete the dataframe
    df_cities_weather.loc[counter] = [id, city, float(lat), float(lon), mean(temp), mean(clear_weather)]

In [64]:
df_cities_weather

,city_id,city,latitude,longitude,temperature,clear_weather
0,156094680,Mont Saint Michel,48.635954,-1.511460,18.954286,33.428571
1,297756747,St Malo,48.649518,-2.026041,18.232857,32.285714
2,297981358,Bayeux,49.276462,-0.702474,18.614286,46.571429
3,298137491,Le Havre,49.493898,0.107973,17.874286,38.857143
4,297518815,Rouen,49.440459,1.093966,18.741429,36.571429
5,297417241,Paris,48.858890,2.320041,19.277143,25.857143
6,297534793,Amiens,49.894171,2.295695,18.958571,28.571429
7,297472400,Lille,50.636565,3.063528,18.481429,27.428571
8,297508568,Strasbourg,48.584614,7.750713,19.581429,15.714286
9,120791766,Chateau du Haut Koenigsbourg,48.249490,7.344296,17.237143,9.714286


In [65]:
fig = px.scatter_mapbox(
    df_cities_weather,
    lat = "latitude",
    lon = "longitude",
    color = "temperature",
    hover_name = "city",
    size = "clear_weather",
    zoom = 3.9,
    mapbox_style = "carto-positron",
    width = 800,
    title = "destination depending on the weather and temperature")
fig.show()

In [66]:
df_cities_weather.to_csv("src/cities_weather.csv")

## Scrape Booking.com

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com

You can scrap as many information asyou want, but we suggest that you get at least:

hotel name,
Url to its booking.com page,
Its coordinates: latitude and longitude
Score given by the website users
Text description of the hotel

In [28]:
!python bookingHotel.py

2022-10-14 12:14:03 [scrapy.utils.log] INFO: Scrapy 2.6.3 started (bot: scrapybot)
2022-10-14 12:14:03 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.19043-SP0
2022-10-14 12:14:03 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-10-14 12:14:03 [scrapy.extensions.telnet] INFO: Telnet Password: cb83a6e1c601b025
2022-10-14 12:14:03 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-10-14 12:14:03 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy

In [67]:
df_booking = pd.read_json("src/booking_hotel.json", encoding='utf-8')
df_booking.head()

,url,city_id,name_hotel,score,coordinates,text_description
0,https://www.booking.com/hotel/fr/express-by-ho...,297534793,"Holiday Inn Express Amiens, an IHG Hotel","7,9","2.27981597007699,49.8737107813327,2.3356917518...","[Rénové en avril 2015, le Holiday Inn Express ..."
1,https://www.booking.com/hotel/fr/cmg-new-apart...,297417241,CMG - New Apartement Montmartre 4P 2BR - 2,None,"2.31625185210765,48.8617907834269,2.3709885478...",[Le CMG - New Apartement Montmartre 4P 2BR - 2...
2,https://www.booking.com/hotel/fr/residhotel-li...,297472400,Residhotel Lille Vauban,"7,7","3.01831070505302,50.6142418000715,3.0750619975...",[Le Residhotel Lille Vauban est idéalement ins...
3,https://www.booking.com/hotel/fr/arc-en-ciel-h...,297504747,Hotel Arc-En-Ciel Colmar Contact Hotel,"8,0","7.33374282489442,48.0694885745119,7.3876310907...",[L'Hotel Arc-En-Ciel Colmar Contact Hotel vous...
4,https://www.booking.com/hotel/fr/hotellitterai...,297518815,Best Western Plus Hotel Litteraire Gustave Fla...,"8,5","1.0589216728517,49.4250988141913,1.11428453728...",[Vous pouvez bénéficier d'une réduction Genius...


Join columns from df_booking with df_weather by the id city

In [68]:
df_booking_weather = df_booking.merge(df_cities_weather, left_on='city_id', right_on='city_id', how='outer')
df_booking_weather.head()

,url,city_id,name_hotel,score,coordinates,text_description,city,latitude,longitude,temperature,clear_weather
0,https://www.booking.com/hotel/fr/express-by-ho...,297534793,"Holiday Inn Express Amiens, an IHG Hotel","7,9","2.27981597007699,49.8737107813327,2.3356917518...","[Rénové en avril 2015, le Holiday Inn Express ...",Amiens,49.894171,2.295695,18.958571,28.571429
1,https://www.booking.com/hotel/fr/le-cange.fr.html,297534793,LE CANGE,"9,5","2.2784373835015,49.8784185834269,2.33431861649...","[Doté d'un jardin et d'une terrasse, l'établis...",Amiens,49.894171,2.295695,18.958571,28.571429
2,https://www.booking.com/hotel/fr/cocon-perche-...,297534793,"Cocon perché, duplex au coeur de la ville","9,0","2.28054049369682,49.8747738834269,2.3364175063...","[Le Cocon perché, duplex au coeur de la ville ...",Amiens,49.894171,2.295695,18.958571,28.571429
3,https://www.booking.com/hotel/fr/grand-appart-...,297534793,GRAND APPART TERRASSE PARKING DESIGN,None,"2.25172627866827,49.8665065834269,2.3075937213...",[GRAND APPART TERRASSE PARKING DESIGN in Amien...,Amiens,49.894171,2.295695,18.958571,28.571429
4,https://www.booking.com/hotel/fr/le-toit-des-f...,297534793,Le toit des Fédérés,"8,0","2.25464950174135,49.8785597834269,2.3105308982...",[Vous pouvez bénéficier d'une réduction Genius...,Amiens,49.894171,2.295695,18.958571,28.571429


In [69]:
df_booking_weather.describe(include="all")

,url,city_id,name_hotel,score,coordinates,text_description,city,latitude,longitude,temperature,clear_weather
count,875,8.750000e+02,875,826,875,875,875,875.000000,875.000000,875.000000,875.000000
unique,875,NaN,874,43,868,875,35,NaN,NaN,NaN,NaN
top,https://www.booking.com/hotel/fr/express-by-ho...,NaN,Le Valmer,"8,5","1.33981155551139,44.0007621834269,1.3898692444...","[Rénové en avril 2015, le Holiday Inn Express ...",Amiens,NaN,NaN,NaN,NaN
freq,1,NaN,2,55,2,1,25,NaN,NaN,NaN,NaN
mean,NaN,2.823779e+08,NaN,NaN,NaN,NaN,NaN,45.840986,3.395928,21.444939,35.191837
std,NaN,5.137416e+07,NaN,NaN,NaN,NaN,NaN,2.554274,2.913222,2.313759,12.925111
min,NaN,7.603631e+07,NaN,NaN,NaN,NaN,NaN,42.525050,-2.026041,17.237143,5.000000
25%,NaN,2.975047e+08,NaN,NaN,NaN,NaN,NaN,43.494514,1.354999,19.277143,25.714286
50%,NaN,2.977491e+08,NaN,NaN,NaN,NaN,NaN,45.187560,4.360069,21.560000,38.857143
75%,NaN,2.980113e+08,NaN,NaN,NaN,NaN,NaN,48.584614,5.735782,23.591429,45.428571


In [70]:
# transforms the score type to integer instead of characters
df_booking_weather['score'] = df_booking_weather['score'].str.replace(",", ".")
df_booking_weather['score'] = df_booking_weather['score'].astype(float)
# calculates latitude and longitude coordinates
df_booking_weather['coordinates'] = df_booking_weather['coordinates'].apply(lambda x : x.split(","))
df_booking_weather['long_hotel'] = df_booking_weather['coordinates'].apply(lambda x : mean([float(x[0]),float(x[2])]))
df_booking_weather['lat_hotel'] = df_booking_weather['coordinates'].apply(lambda x : mean([float(x[1]),float(x[3])]))
# transforms text_description into a string instead of a list
df_booking_weather['text_description'] = df_booking_weather['text_description'].apply(lambda x : " ".join(x))
# drop the old column
df_booking_weather.drop(columns=['coordinates'], inplace = True)

In [71]:
df_booking_weather.head()

,url,city_id,name_hotel,score,text_description,city,latitude,longitude,temperature,clear_weather,long_hotel,lat_hotel
0,https://www.booking.com/hotel/fr/express-by-ho...,297534793,"Holiday Inn Express Amiens, an IHG Hotel",7.9,"Rénové en avril 2015, le Holiday Inn Express A...",Amiens,49.894171,2.295695,18.958571,28.571429,2.307754,49.891676
1,https://www.booking.com/hotel/fr/le-cange.fr.html,297534793,LE CANGE,9.5,"Doté d'un jardin et d'une terrasse, l'établiss...",Amiens,49.894171,2.295695,18.958571,28.571429,2.306378,49.896384
2,https://www.booking.com/hotel/fr/cocon-perche-...,297534793,"Cocon perché, duplex au coeur de la ville",9.0,"Le Cocon perché, duplex au coeur de la ville p...",Amiens,49.894171,2.295695,18.958571,28.571429,2.308479,49.892739
3,https://www.booking.com/hotel/fr/grand-appart-...,297534793,GRAND APPART TERRASSE PARKING DESIGN,NaN,GRAND APPART TERRASSE PARKING DESIGN in Amiens...,Amiens,49.894171,2.295695,18.958571,28.571429,2.279660,49.884472
4,https://www.booking.com/hotel/fr/le-toit-des-f...,297534793,Le toit des Fédérés,8.0,Vous pouvez bénéficier d'une réduction Genius ...,Amiens,49.894171,2.295695,18.958571,28.571429,2.282590,49.896525


In [72]:
df_booking.to_csv("src/booking_weather.csv")

Top-5 destinations

In [73]:
top_5_cities = df_cities_weather.sort_values(by = 'clear_weather', ascending = False).head(5)[["city_id","city"]]
top_5_cities

,city_id,city
16,297681191,Lyon
14,298516920,Annecy
26,298011281,Saintes Maries de la mer
22,297768703,Avignon
15,297639071,Grenoble


In [74]:
for id , city in top_5_cities.values:
     mask = df_booking_weather["city_id"] == id
     df = df_booking_weather.loc[mask,:].head(20)
     fig = px.scatter_mapbox(
          df,
          lat = "lat_hotel",
          lon = "long_hotel",
          color = "score",
          hover_name = "name_hotel",
          hover_data = ['url'],
          zoom = 8,
          mapbox_style = "open-street-map",
          width = 800,
          title = f'{city}')
     fig.show()
     

Create a data lake using S3

In [37]:
from credentials import MY_ACCESS_KEY_ID, MY_SECRET_ACCESS_KEY

# create a Boto3 session
session = boto3.Session(aws_access_key_id = MY_ACCESS_KEY_ID, 
                        aws_secret_access_key = MY_SECRET_ACCESS_KEY)

In [38]:
# create a resource session
s3 = session.resource("s3")

In [95]:
# Create a Bucket 
bucket_booking =s3.create_bucket(Bucket="booking-cities-weather")

In [97]:
# to export your DataFrame as a csv file
csv = df_booking_weather.to_csv()

In [108]:
# put_object() function to create an Object within the bucket 
put_object = bucket_booking.put_object(Key="booking_weather_s3.csv", Body=csv)

create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB.

In [42]:
# Create an sqlalchemy engine that is connected to your AWS RDS instance

from credentials import USERNAME_DB, PASSWORD_DB, HOSTNAME_DB

USERNAME = USERNAME_DB
PASSWORD = PASSWORD_DB
HOSTNAME = HOSTNAME_DB

engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/postgres", echo=True)

In [109]:
for my_bucket_object in bucket_booking.objects.all(): # list all files in bucket
    print(my_bucket_object.key)

booking_weather.csv
booking_weather_s3.csv


In [111]:
# extract your data from S3
bucket_booking.download_file('booking_weather.csv', 'src/booking_weather_s3.csv')

In [112]:
df = pd.read_csv('src/booking_weather_s3.csv', index_col=0)
# sCreate a table in your remote database containing the information of the dataset
df.to_sql(
    "booking_cities_weather",
    engine
)

2022-10-14 13:20:16,757 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-10-14 13:20:16,758 INFO sqlalchemy.engine.Engine [cached since 3384s ago] {'name': 'booking_cities_weather'}
2022-10-14 13:20:16,847 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-14 13:20:16,850 INFO sqlalchemy.engine.Engine 
CREATE TABLE booking_cities_weather (
	index BIGINT, 
	url TEXT, 
	city_id BIGINT, 
	name_hotel TEXT, 
	score FLOAT(53), 
	text_description TEXT, 
	city TEXT, 
	latitude FLOAT(53), 
	longitude FLOAT(53), 
	temperature FLOAT(53), 
	clear_weather FLOAT(53), 
	long_hotel FLOAT(53), 
	lat_hotel FLOAT(53)
)


2022-10-14 13:20:16,851 INFO sqlalchemy.engine.Engine [no key 0.00098s] {}
2022-10-14 13:20:16,939 INFO sqlalchemy.engine.Engine CREATE INDEX ix_booking_cities_weather_index ON booking_cities_weather (index)
2022-10-14 13:20:16,941 INFO sqlalchemy.engine.Engin

875

In [113]:
# Create a statement 
# Within the text() method is a SQL query. Check out our SQL reminder course if you feel a little rusty
stmt = text("SELECT DISTINCT city FROM booking_cities_weather")

# Read a dataFrame using SQL

df_cities = pd.read_sql(
        stmt,
        engine
    )               

df_cities

2022-10-14 13:21:25,560 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-10-14 13:21:25,564 INFO sqlalchemy.engine.Engine [cached since 3452s ago] {'name': 'SELECT DISTINCT city FROM booking_cities_weather'}
2022-10-14 13:21:25,676 INFO sqlalchemy.engine.Engine SELECT DISTINCT city FROM booking_cities_weather
2022-10-14 13:21:25,679 INFO sqlalchemy.engine.Engine [cached since 2217s ago] {}


,city
0,Bayonne
1,Aigues Mortes
2,Strasbourg
3,Toulouse
4,La Rochelle
5,Uzes
6,Mont Saint Michel
7,Paris
8,Avignon
9,Aix en Provence
